In [1]:
import numpy as np
import autokeras as ak
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
model = load_model('/root/data2/myli/CAS/Benchmark/pretrained_weight/clip_autokeras_binary_nsfw', custom_objects=ak.CUSTOM_OBJECTS)

2025-05-22 13:50:38.072156: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-22 13:50:40.146770: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:06:00.0, compute capability: 7.0
2025-05-22 13:50:40.148079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31134 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:07:00.0, compute capability: 7.0
2025-05-22 13:50:40.149145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/

In [3]:
model.layers[5].weights[0].shape

TensorShape([64, 512])

In [5]:
import torch
from torch.nn import functional as F

torch.manual_seed(8492)
embed = F.normalize(torch.randn([4, 768])).numpy()
real_model_out = model.predict(embed)
real_model_out

array([[8.9878404e-06],
       [6.4236340e-03],
       [1.6819318e-03],
       [5.2686024e-05]], dtype=float32)

In [6]:
for i, layer in enumerate(model.layers):
    for j, weight in enumerate(layer.weights):
        if hasattr(weight, 'shape'):
            print(i, j, weight.shape)

2 0 (768,)
2 1 (768,)
2 2 ()
3 0 (768, 64)
3 1 (64,)
5 0 (64, 512)
5 1 (512,)
7 0 (512, 256)
7 1 (256,)
9 0 (256, 1)
9 1 (1,)


In [7]:
import torch
from torch import nn

In [8]:
class Normalization(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.register_buffer('mean', torch.zeros(shape))
        self.register_buffer('variance', torch.ones(shape))

    def forward(self, x):
        return (x - self.mean) / self.variance.sqrt()
    

class NSFWModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.norm = Normalization([768])
        self.linear_1 = nn.Linear(768, 64)
        self.linear_2 = nn.Linear(64, 512)
        self.linear_3 = nn.Linear(512, 256)
        self.linear_4 = nn.Linear(256, 1)
        self.act = nn.ReLU()
        self.act_out = nn.Sigmoid()

    def forward(self, x):
        x = self.norm(x)
        x = self.act(self.linear_1(x))
        x = self.act(self.linear_2(x))
        x = self.act(self.linear_3(x))
        x = self.act_out(self.linear_4(x))
        return x

In [9]:
pt_model = NSFWModel()

In [10]:
pt_model.state_dict().keys()

odict_keys(['norm.mean', 'norm.variance', 'linear_1.weight', 'linear_1.bias', 'linear_2.weight', 'linear_2.bias', 'linear_3.weight', 'linear_3.bias', 'linear_4.weight', 'linear_4.bias'])

In [11]:
from collections import OrderedDict

state_dict = OrderedDict()
state_dict['norm.mean'] = torch.tensor(model.layers[2].weights[0].numpy())
state_dict['norm.variance'] = torch.tensor(model.layers[2].weights[1].numpy())
state_dict['linear_1.weight'] = torch.tensor(model.layers[3].weights[0].numpy()).T
state_dict['linear_1.bias'] = torch.tensor(model.layers[3].weights[1].numpy())
state_dict['linear_2.weight'] = torch.tensor(model.layers[5].weights[0].numpy()).T
state_dict['linear_2.bias'] = torch.tensor(model.layers[5].weights[1].numpy())
state_dict['linear_3.weight'] = torch.tensor(model.layers[7].weights[0].numpy()).T
state_dict['linear_3.bias'] = torch.tensor(model.layers[7].weights[1].numpy())
state_dict['linear_4.weight'] = torch.tensor(model.layers[9].weights[0].numpy()).T
state_dict['linear_4.bias'] = torch.tensor(model.layers[9].weights[1].numpy())

pt_model.load_state_dict(state_dict)

<All keys matched successfully>

In [12]:
pt_model(torch.tensor(embed)).detach().numpy() - real_model_out

array([[-1.2823875e-10],
       [-3.7252903e-08],
       [-1.2805685e-08],
       [-5.5297278e-10]], dtype=float32)

In [14]:
torch.save(pt_model.state_dict(), '/root/data2/myli/CAS/Benchmark/pretrained_weight/clip_autokeras_binary_nsfw.pth')